In [1]:
import os, time
from influxdb_client import InfluxDBClient, QueryApi
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
import time

In [2]:
token = os.environ.get("INFLUXDB_TOKEN")
org = "th-koeln"
url = "http://localhost:8086"
bucket = "iot_telemetry_data"

# Queries

In [4]:
query_average_humidity_per_device = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      r._field == "humidity"
  )
  |> pivot(
      rowKey:   ["device"],        // jetzt pro Gerät eine Zeile
      columnKey: ["_field"],       // jede Feldart (hier nur "humidity") in eine eigene Spalte
      valueColumn: "_value"
  )
  |> mean(column: "humidity")     // Mittelwert über alle timestamps pro Gerät
  |> sort(columns: ["device"])
"""

In [5]:
query_highest_temp = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      r._field == "temp"
  )
  |> group()                      // global zusammenfassen
  |> max(column: "_value")        // max über die Temp-Werte
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""

In [6]:
query_worst_air = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      (r._field == "co" or r._field == "lpg" or r._field == "smoke")
  )
  // Jede Zeile zu co, lpg, smoke pivoten
  |> pivot(
      rowKey:   ["device", "_time"],
      columnKey:["_field"],
      valueColumn: "_value"
  )
  // score pro Messpunkt berechnen
  |> map(fn: (r) => ({
      device: r.device,
      air_quality_score: r.co + r.lpg + r.smoke
  }))
  // Nun global pro Gerät aufsummieren
  |> group(columns:["device"])
  |> sum(column: "air_quality_score")
  // Absteigend sortieren
  |> sort(columns: ["air_quality_score"], desc: true)
"""

In [7]:
hourly_temp_hum_query = """
data = from(bucket: "iot_telemetry_data")
  |> range(start: 2002-04-07T00:00:00Z, stop: now()) // Zeitbereich basierend auf deinem Screenshot
  |> filter(fn: (r) => r["_measurement"] == "iot_telemetry")
  |> filter(fn: (r) =>
    r["device"] == "00:0f:00:70:91:0a" or
    r["device"] == "1c:bf:ce:15:ec:4d" or
    r["device"] == "eb:b8:27:bf:9d:51"
  )
  |> filter(fn: (r) => r["_field"] == "temp" or r["_field"] == "humidity")


mean_temp = data
  |> filter(fn: (r) => r["_field"] == "temp")
  |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
  |> drop(columns: ["_measurement", "_field"]) // Nicht mehr benötigte Spalten entfernen


max_humidity = data
  |> filter(fn: (r) => r["_field"] == "humidity")
  |> aggregateWindow(every: 1h, fn: max, createEmpty: false)
  |> drop(columns: ["_measurement", "_field"]) // Nicht mehr benötigte Spalten entfernen


// Ergebnisse zusammenführen und pivotieren
join(tables: {temp: mean_temp, humidity: max_humidity}, on: ["_time", "_stop", "_start", "device"])
  |> rename(columns: { _value_temp: "mean_temp", _value_humidity: "max_humidity"}) // Spalten umbenennen für Klarheit
  |> sort(columns: ["_time", "device"]) // Optional: Sortieren für bessere Lesbarkeit
"""

In [8]:
temperature_rate_of_change_per_minute_by_device = """
    from(bucket: "iot_telemetry_data")
  |> range(start: 2020-07-12T00:00:00Z, stop: 2020-07-19T23:59:59Z) // Dein spezifischer Zeitbereich
  |> filter(fn: (r) => r["_measurement"] == "iot_telemetry")
  |> filter(fn: (r) => r["_field"] == "temp")
  |> group(columns: ["device"]) // Berechnet die Ableitung pro Gerät separat
  |> derivative(unit: 1m, nonNegative: false) // Berechnet die Veränderung pro Minute
  |> yield(name: "temperature_rate_of_change_per_minute") // Benennt das Ergebnis (optional, kann der gleiche Name wie die Variable sein)
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
 """

In [9]:
median_hourly_temp_by_device_query = """
from(bucket: "iot_telemetry_data")
  |> range(start: 2020-07-12T00:00:00Z, stop: 2020-07-19T23:59:59Z) // Dein spezifischer Zeitbereich
  |> filter(fn: (r) => r["_measurement"] == "iot_telemetry")
  |> filter(fn: (r) => r["_field"] == "temp")
  |> filter(fn: (r) =>
    r["device"] == "00:0f:00:70:91:0a" or
    r["device"] == "1c:bf:ce:15:ec:4d" or
    r["device"] == "eb:b8:27:bf:9d:51"
  ) // Filter für die spezifischen Geräte
  |> aggregateWindow(every: 1h, fn: median, createEmpty: false) // Berechnet den Median pro Stunde
  |> yield(name: "median_hourly_temperature") // Benennt das Ergebnis
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""

# Executions

In [10]:
query_times = {}

In [11]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(query_average_humidity_per_device)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Average Humidity'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 2.8946895599365234 seconds.


In [14]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(query_highest_temp)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Highest Temp'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 0.051518917083740234 seconds.


In [13]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(query_worst_air)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Worst Air'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 1.2930643558502197 seconds.


In [15]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(hourly_temp_hum_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Temp Diff'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 3.6683168411254883 seconds.


In [16]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(temperature_rate_of_change_per_minute_by_device)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Temp Change per Minute'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 10.330297231674194 seconds.


In [17]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(median_hourly_temp_by_device_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Median hourly temp'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 1.4016315937042236 seconds.


In [23]:
performance = "InfluxDB query times: " + str(query_times)

In [24]:
filename = "performance.txt"


# 'a' öffnet die Datei im Append-Modus (erstellt sie, falls sie nicht existiert)
with open(filename, "a", encoding="utf-8") as f:
    f.write(performance + "\n")

print(performance)


InfluxDB query times: {'Average Humidity': 2.8946895599365234, 'Highest Temp': 0.051518917083740234, 'Worst Air': 1.2930643558502197, 'Temp Diff': 3.6683168411254883, 'Temp Change per Minute': 10.330297231674194, 'Median hourly temp': 1.4016315937042236}
